In [ ]:
print("------------------")
print("Program Started for DNN Char Yield Model")
print("------------------\n")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics, model_selection, neighbors, svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import os

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)


# Loading Data 

In [ ]:

dir_p = 'C:\\Users\\Honeyz\\Desktop\\Modeling\\Final models\\CoPyro Project\\Data\\CharCoPyroDataProcessed_Proximate.csv'

raw_dataset = pd.read_csv(dir_p, skipinitialspace=True)

dataset = raw_dataset.copy()
dataset = dataset.dropna()
print("--------Data have been loaded & splitted---------\n")


# Pearson Corr

In [ ]:
pearsoncorr = dataset.corr(method='pearson')
f, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(pearsoncorr, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True


heatmap = sns.heatmap(pearsoncorr,
                      square = True,
                      mask=mask,
                      linewidths = .5,
                      cmap = 'coolwarm',
                      cbar_kws = {'shrink': .4,
                                'ticks' : [-1, -.5, 0, 0.5, 1]},
                      vmin = -1,
                      vmax = 1,
                      annot = True,
                      annot_kws = {'size': 12})

#add the column names as labels
ax.set_yticklabels(pearsoncorr.columns, rotation = 0)
ax.set_xticklabels(pearsoncorr.columns)

sns.set_style({'xtick.bottom': True}, {'ytick.left': True})


#### Peason Correlation only capture linear dependancies. 


# Data Split and stratification 

In [ ]:
_seed = 42 
_random_state = np.random.RandomState(_seed)
strata_1 = pd.cut(dataset.loc[:, "w%P"], bins=[-1, 15, 35, 50, 75, 90, np.inf],labels=[1, 2, 3, 4, 5, 6])
training_data, testing_data = model_selection.train_test_split(dataset, test_size=0.1, stratify=strata_1, random_state=_random_state)
print("--------Data have been stratified----------\n")


#### Data is stratified with polymer weight percent in the sample to have samiliar distribution in the test and train samples. This prevents the random split from accidenatly allocate test data out of the training range. 

In [ ]:
_ = (testing_data.loc[:, "w%P"]
                 .hist())
plt.show()
__ = (dataset.loc[:, "w%P"]
               .hist())

# Data preprocessing 

In [ ]:
training_features = training_data.drop("Char%", axis=1)
training_target = training_data.loc[:, ["Char%"]]

testing_features = testing_data.drop("Char%", axis=1)
testing_target = testing_data.loc[:, ["Char%"]]
print("--------Data have been divided into features and targets----------\n")

In [ ]:

sc = StandardScaler()
mmc = MinMaxScaler()
X_train = mmc.fit_transform(training_features)
X_test = mmc.fit_transform(testing_features)

#### Scaling the data helps the model converge more better

# Training and Cross Validation

## Simple random forest

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(training_features, training_target)

predictions = regressor.predict(testing_features)
mae = metrics.mean_absolute_error(testing_target, predictions)
mse = metrics.mean_squared_error(testing_target, predictions)
rmse = mse**0.5

print(rmse)
print(mae)

## Cross Validation with Random Forsest

In [ ]:
RandomForest_regression_scores = model_selection.cross_val_score(RandomForestRegressor(),
                                                           X=training_features,
                                                           y=training_target,
                                                           cv=10,
                                                           scoring="neg_mean_absolute_error",
                                                           n_jobs=-1)
def display_mae(rmses):
    print("MAE mean:", rmses.mean())
    print("MAE standard deviation:", rmses.std())

RandomForest_regression_mae = (-RandomForest_regression_scores)
display_mae(RandomForest_regression_mae)

## Cross Validation with K-nearest nieghbor model


In [ ]:
knn_scores = model_selection.cross_val_score(neighbors.KNeighborsRegressor(),
                                             X=training_features,
                                             y=training_target,
                                             cv=10,
                                             scoring="neg_mean_absolute_error",
                                             n_jobs=-1)

knn_mae = (-knn_scores)
display_mae(knn_mae)

## Cross Validation with Support Vector Machine model


In [ ]:
svr_scores = model_selection.cross_val_score(svm.SVR(),
                                             X=training_features,
                                             y=training_target,
                                             cv=10,
                                             scoring="neg_mean_absolute_error",
                                             n_jobs=-1)

svr_mae = (-svr_scores)
display_mae(svr_mae)

# Training and hyperparamers search with Cross Validation

## XGBoost with Random search to identify the scale of best hyperparameters

In [ ]:
from scipy import stats

_param_distributions = {
    "n_estimators": stats.geom(p=0.01),
    "min_samples_split": stats.beta(a=1, b=99),
    "min_samples_leaf": stats.beta(a=1, b=999),
}

# Hyperparameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 4000, stop = 10000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print("--------Model and search space have been defined----------\n")
_random_state = 42
xgb_regressor = XGBRegressor(random_state=_random_state)

randomized_search_cv = model_selection.RandomizedSearchCV(
    xgb_regressor,
    param_distributions=_param_distributions,
    scoring="neg_mean_absolute_error",
    random_state=42,
    n_iter=10,
    cv=10,
    n_jobs=-1,
    verbose=10
)

randomized_search_cv.fit(training_features, training_target)
best_random = randomized_search_cv.best_estimator_
rsbs = randomized_search_cv.best_params_
print("--------------------------------")


print(rsbs)
print("--------------------------------")

# RMSE for the best parameters
print(-randomized_search_cv.best_score_)

predictions = best_random.predict(testing_features)
mae = metrics.mean_absolute_error(testing_target, predictions)
mse = metrics.mean_squared_error(testing_target, predictions)
rmse = mse**0.5
mse = metrics.mean_squared_error(testing_target, predictions)
r2 = metrics.r2_score(testing_target, predictions)
print("--------------------------------")
print(rmse)
print(r2)

## XGBoost with G=grid search to locate the scale of best hyperparameters

In [ ]:

# Hyperparameters

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

_random_state = 42
xgb_regressor = XGBRegressor(random_state=_random_state)

Grid_Search_cv = model_selection.GridSearchCV(
    xgb_regressor,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=10,
    n_jobs=-1,
    verbose=10
)

Grid_Search_cv.fit(training_features, training_target)
best_grid = Grid_Search_cv.best_estimator_
gsbs = Grid_Search_cv.best_params_
print("--------------------------------")
print("Model Best parameters.........\n")

print(gsbs)
print("--------------------------------")

# RMSE for the best parameters
print(-Grid_Search_cv.best_score_)

predictions = best_grid.predict(testing_features)
mae = metrics.mean_absolute_error(testing_target, predictions)
mse = metrics.mean_squared_error(testing_target, predictions)
rmse = mse**0.5
mse = metrics.mean_squared_error(testing_target, predictions)
r2 = metrics.r2_score(testing_target, predictions)

print("--------------------------------")
print("--------Test the model on hideout test data----------\n")
print("The RMSE score for the test :", rmse)
print("The R^2 score for the test :", r2)
print("The RMSE score for the test :", mae)

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(testing_target, predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
OutF = open("C:\\Users\\Honeyz\\Desktop\\Modeling\\Final models\\CoPyro Project\\Results\\Performace_Output_Char_RF_Notebook.txt","a")
OutF.write("\n RMSE: ")
OutF.write(str(round(rmse,3)))
OutF.write("\n MAE: ")
OutF.write(str(round(mae,3)))
OutF.write("\n RSBS: ")
OutF.write(str(rsbs))
OutF.write("\n")
OutF.close()

In [ ]:
OutTD = open("C:\\Users\\Honeyz\\Desktop\\Modeling\\Final models\\CoPyro Project\\Results\\TestData_Output_Char_RF_Notebook.txt","a")
OutTD.write("\n Actual data: ")
OutTD.write(str(testing_target.values))
OutTD.write("\n Prediction data: ")
OutTD.write(str(predictions))
OutTD.close()

In [ ]:
# save the model
filename = 'C:\\Users\\Honeyz\\Desktop\\Modeling\\Final models\\CoPyro Project\\Saved Models\\RF_Char_Model_Proximate_3.sav'
pickle.dump(regressor, open(filename, 'wb'))

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)